In [9]:
import pandas as pd
from census import Census
# Get Variable Code
def get_acs_code(var_select,filepath = '../data/acs_variable_code.csv'):
    acs_code_df = pd.read_csv(filepath)
    acs_dict = {}
    for i in range(len(acs_code_df)):
        acs_dict[acs_code_df.loc[i,'Variable Name']] = acs_code_df.loc[i,'Variable Code']

    code_list = ['NAME']
    for f in var_select:
        code_list.append(acs_dict[f])
    return tuple(code_list), acs_dict

code_list,acs_dict = get_acs_code(var_select=['Total Population', 'Total Housing Units'],filepath = '../data/acs_variable_code.csv')
print(code_list)
import requests
def get_fips_code(lat, lon):
    url = "https://geo.fcc.gov/api/census/block/find"
    params = {
        "latitude": lat,
        "longitude": lon,
        "format": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    state_fips = data["State"]["FIPS"]
    state_name = data["State"]["name"]
    county_fips = data["County"]["FIPS"]
    county_name = data["County"]["name"]
    return state_fips,state_name, county_fips, county_name

def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

api_file = '/Users/caoguangyue/GitHub/Urban-Design-Equity-Metrics/app/census_api_key.txt'
api_key = get_file_contents(api_file)
c = Census(api_key)
# Replace with Input from lat, lng
state_fips, state_name, county_fips, county_name = get_fips_code(37.7749, -122.4194)
print(code_list)
print(state_fips, county_fips)


sc_census = c.acs5.state_county_blockgroup(fields= code_list, 
                                           state_fips = str(state_fips), 
                                           county_fips = str(county_fips[2::]), 
                                           blockgroup = '*', 
                                           year = 2019)

#### 2. Feature engineering
# Feature Engineering
acs_df = pd.DataFrame(sc_census)
acs_df['GEOID'] = acs_df['state'] + acs_df['county'] + acs_df['tract'] + acs_df['block group']


inv_acs_dict = {v: k for k, v in acs_dict.items()}
acs_df.rename(columns=inv_acs_dict, inplace=True)

acs_df.head()

('NAME', 'B01003_001E', 'B25001_001E')
('NAME', 'B01003_001E', 'B25001_001E')
06 06075


,NAME,Total Population,Total Housing Units,state,county,tract,block group,GEOID
0,"Block Group 4, Census Tract 452, San Francisco...",1029.0,469.0,06,075,045200,4,060750452004
1,"Block Group 2, Census Tract 452, San Francisco...",1838.0,892.0,06,075,045200,2,060750452002
2,"Block Group 5, Census Tract 452, San Francisco...",1493.0,539.0,06,075,045200,5,060750452005
3,"Block Group 1, Census Tract 111, San Francisco...",1755.0,1296.0,06,075,011100,1,060750111001
4,"Block Group 1, Census Tract 117, San Francisco...",859.0,586.0,06,075,011700,1,060750117001


In [14]:
acs_df.dtypes

NAME                    object
Total Population       float64
Total Housing Units    float64
state                   object
county                  object
tract                   object
block group             object
GEOID                   object
dtype: object

In [3]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pygris

In [6]:
state_name

'California'

In [7]:
county_tiger = pygris.block_groups(state = state_name, county = county_name, cache = True, year = 2019)


Using FIPS code '06' for input 'California'
Using FIPS code '075' for input 'San Francisco County'


In [15]:
county_tiger.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
771,06,075,012201,2,060750122012,Block Group 2,G5030,S,30528,0,+37.7849116,-122.4161659,"POLYGON ((-122.41790 37.78517, -122.41626 37.7..."
772,06,075,012401,2,060750124012,Block Group 2,G5030,S,45978,0,+37.7831562,-122.4149685,"POLYGON ((-122.41607 37.78445, -122.41478 37.7..."
773,06,075,012402,3,060750124023,Block Group 3,G5030,S,62826,0,+37.7830940,-122.4192115,"POLYGON ((-122.42107 37.78382, -122.42093 37.7..."
774,06,075,012602,1,060750126021,Block Group 1,G5030,S,248192,544217,+37.8064743,-122.4356584,"POLYGON ((-122.44308 37.80973, -122.43008 37.8..."
775,06,075,012901,1,060750129011,Block Group 1,G5030,S,90524,0,+37.8010280,-122.4270595,"POLYGON ((-122.42967 37.80165, -122.42802 37.8..."


In [13]:
county_tiger.dtypes

STATEFP       object
COUNTYFP      object
TRACTCE       object
BLKGRPCE      object
GEOID         object
NAMELSAD      object
MTFCC         object
FUNCSTAT      object
ALAND          int64
AWATER         int64
INTPTLAT      object
INTPTLON      object
geometry    geometry
dtype: object

In [20]:
acs_gdf = acs_df.merge(county_tiger, on='GEOID', how='inner')
acs_gdf = acs_gdf.set_geometry('geometry')

In [21]:
acs_gdf.explore()